In [1]:
import time
import numpy as np
import torch
torch.autograd.set_grad_enabled(False)
# torch.cuda.set_device(0)

In [11]:
!git clone --depth 1 https://github.com/Hzzone/torch_clustering
import sys
sys.path.insert(0, './torch_clustering')
!pip install munkres faiss-gpu

fatal: destination path 'torch_clustering' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 122 kB/s 


In [2]:
!nvidia-smi

Wed Oct 19 09:12:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from torch_clustering import PyTorchKMeans, FaissKMeans, PyTorchGaussianMixture, evaluate_clustering

In [ ]:
# CIFAR-10 features 256d
# https://drive.google.com/file/d/1N3REtPr-A__KOtzRZenFSvRIEufW9j3A/view
# ImageNet features 256d
# https://drive.google.com/file/d/1UcUWw9o6Y6SrSFXOkNTVjCoEik79oUAo/view

In [5]:
!gdown --id 1N3REtPr-A__KOtzRZenFSvRIEufW9j3A -O cifar10_features.pth
# !gdown --id 1UcUWw9o6Y6SrSFXOkNTVjCoEik79oUAo -O imagenet_features.pth

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1N3REtPr-A__KOtzRZenFSvRIEufW9j3A
To: /content/cifar10_features.pth
100% 51.4M/51.4M [00:00<00:00, 57.7MB/s]


In [4]:
features_path = './cifar10_features.pth'
# features_path = '.imagenet_features.pth'
data = torch.load(features_path, map_location='cpu')
features, labels = data[:, :-1], data[:, -1]
features, labels = features.cuda().float(), labels.cuda().long()
features_numpy, labels_numpy = features.cpu().numpy(), labels.cpu().numpy()

In [5]:
kwargs = {
    'metric': 'cosine',  # euclidean if not l2_normalize
    'distributed': False,
    'random_state': 0,
    'n_clusters': int(labels_numpy.max() + 1),
    'verbose': False
}

In [9]:
clustering_model = PyTorchKMeans(init='k-means++', max_iter=300, tol=1e-4, **kwargs)
psedo_labels = clustering_model.fit_predict(features)
evaluate_clustering(labels.cpu().numpy(), psedo_labels.cpu().numpy(), eval_metric=['nmi', 'acc'], phase='train')

{'train_nmi': 0.8551917177300482, 'train_acc': 0.92368}

In [6]:
import faiss
clustering_model = FaissKMeans(n_init=5, max_iter=50, **kwargs)
psedo_labels = clustering_model.fit_predict(features_numpy, device=0)
evaluate_clustering(labels.cpu().numpy(), psedo_labels.cpu().numpy(), eval_metric=['nmi', 'acc'], phase='train')

{'train_nmi': 0.855235833704221, 'train_acc': 0.92358}

In [7]:
clustering_model = PyTorchGaussianMixture(covariance_type='diag', reg_covar=1e-6, init='k-means++', **kwargs)
psedo_labels = clustering_model.fit_predict(features)
evaluate_clustering(labels.cpu().numpy(), torch.argmax(psedo_labels, dim=1).cpu().numpy(), eval_metric=['nmi', 'acc'])

{'train_nmi': 0.8559683572879772, 'train_acc': 0.92388}

In [8]:
n_times = 10
all_times = []
for i in range(n_times):
    st = time.time()
    PyTorchKMeans(init='k-means++', max_iter=300, tol=1e-4, **kwargs).fit_predict(features)
    all_times.append(time.time() - st)
np.mean(all_times), np.std(all_times)

(0.7327220916748047, 0.09029591208799001)

In [9]:
all_times = []
for i in range(n_times):
    st = time.time()
    FaissKMeans(n_init=5, max_iter=50, **kwargs).fit_predict(features_numpy, device=0)
    all_times.append(time.time() - st)
np.mean(all_times), np.std(all_times)

(4.598904633522034, 0.05315894215389966)

In [10]:
all_times = []
for i in range(n_times):
    st = time.time()
    PyTorchGaussianMixture(covariance_type='diag', reg_covar=1e-6, init='k-means++', **kwargs).fit_predict(features)
    all_times.append(time.time() - st)
np.mean(all_times), np.std(all_times)

(11.671153950691224, 0.33552815247925466)